In [1]:
import json
import re
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage
from openai import OpenAI

In [2]:
BASE_URL = 'http://vllm:8000/v1'

MODEL_NAME = '/models/Qwen3-8B-FP8'

## 処理の定義

In [3]:
def extract_think_and_others(text: str):
    pattern = re.compile(r"<think>(.*?)</think>", re.DOTALL)
    results = []
    last_end = 0

    for match in pattern.finditer(text):
        # think前のnon_think部分
        non_think = text[last_end:match.start()]
        if non_think.strip():
            results.append({
                "type": "non_think",
                "content": non_think.strip()
            })

        # think部分
        think = match.group(1)
        results.append({
            "type": "think",
            "content": think.strip()
        })

        last_end = match.end()

    # 最後の残り部分
    if last_end < len(text):
        tail = text[last_end:]
        if tail.strip():
            results.append({
                "type": "non_think",
                "content": tail.strip()
            })
    return results


class OpenAIRequest:
    def __init__(
        self,
        base_url: str=BASE_URL,
        chat_args={
            'max_tokens': 8192,
            'temperature': 0.7,
            'top_p': 0.8,
            'presence_penalty': 1.5,
            'extra_body': {
                'top_k': 20,
                'chat_template_kwargs': {'enable_thinking': True}
            },
            'model': MODEL_NAME
        }
    ):
        self.client = OpenAI(
            api_key='EMPTY',
            base_url=base_url
        )
        self.chat_args = chat_args

    def __call__(
        self,
        prompt: str,
    ):
        chat_response = self.client.chat.completions.create(
            messages=[{'role': 'user', 'content': prompt}],
            **self.chat_args
        )
        text = chat_response.choices[0].message.content
        return extract_think_and_others(text)

## curlを使用してみる

In [6]:
%%bash
curl http://vllm:8000/v1/chat/completions -H "Content-Type: application/json" -d '{
  "model": "/models/Qwen3-8B-FP8",
  "messages": [
    {"role": "user", "content": "貴方の名前は？"}
  ],
  "temperature": 0.7,
  "top_p": 0.8,
  "top_k": 20,
  "max_tokens": 8192,
  "presence_penalty": 1.5,
  "chat_template_kwargs": {"enable_thinking": true}
}'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2983  100  2713  100   270    221     22  0:00:12  0:00:12 --:--:--   674


{"id":"chatcmpl-285e1156930d4404afcb916e7c821d16","object":"chat.completion","created":1748264473,"model":"/models/Qwen3-8B-FP8","choices":[{"index":0,"message":{"role":"assistant","reasoning_content":null,"content":"<think>\nOkay, the user asked, \"貴方の名前は？\" which means \"What's your name?\" in Japanese. I need to respond in Japanese since the question is in Japanese.\n\nFirst, I should confirm that my name is Qwen. But I need to translate that into Japanese. The direct translation of Qwen is \"クウェン\" (Kuwen). However, sometimes people might refer to me as \"通義千問\" (Tongyi Qianwen), but since the user asked for my name, it's better to stick with the official name.\n\nWait, the user might be expecting a more natural Japanese response. Maybe I should say something like \"私はQwenです。\" which means \"I am Qwen.\" That's straightforward and clear. Alternatively, using the Japanese katakana for Qwen would be appropriate here. \n\nI should also consider if there's any cultural nuance I'm missi

## OpenAIクライアントを使用してみる

In [7]:
open_ai_request = OpenAIRequest()
result = open_ai_request('英語と日本語どちらが難しい？')
for res in result:
    if res['type'] == 'think':
        continue
    else:
        print(res['content'])

英語と日本語の難しさは、学習者の母語や目的、個人の得意分野によって大きく異なります。以下に両言語の主な特徴と比較をまとめました：

---

### **1. 英語の難しい点**
- **文法・構文**  
  - 非常に複雑で、文型（SVOなど）や時制（現在形・過去形・未来形）、仮定法などのルールが多岐にわたります。  
  - 動詞の活用（例：「to be」の変化、「have」の過去形・過去分詞）や名詞の可算不可算の区別が初学者にはわかりにくいです。  
  - 複数形（-sの付加）、所有格（'s）、冠詞（the/a/an）の使用も難しいポイントです。

- **発音・アクセント**  
  - 発音が不規則（例：th音、r音の発音、アイスクリームの「i」の発音）。  
  - アクセントやリズムが直訳では理解しにくい（例：「I'm going to the store」の重音位置）。

- **語彙・表現**  
  - 多くの同義語や慣用句があり、意味のニュアンスを正確に伝えるのが難しい（例：「break a leg」＝「頑張れ」）。  
  - 現代英語には多くの外来語や新語が含まれるため、学習コストが高い。

- **文化・文脈**  
  - ユーモアやイディオムの理解には文化的背景が必要です。  
  - 会話での非言語的要素（トーン、ジェスチャー）が重要です。

---

### **2. 日本語の難しい点**
- **文字体系**  
  - 漢字（約2,000字以上）の読み書きが必須で、読解力に直結します。  
  - 平仮名・片仮名との組み合わせが複雑（例：「食べる」→「たべる」「たべる」）。  
  - 漢字の表記法（当用漢字・常用漢字）や読み方（音読み・訓読み）の違いがあります。

- **文法・構文**  
  - 文の順序が逆転する傾向（例：「私は本を読む」→ 英語の「I read a book」）。  
  - 助詞（が・は・を・になど）の使い分けが難しい。  
  - 動詞の活用（五段・一段・カ行変格など）や敬語（丁寧語・普通語・謙譲語）の区別が複雑。

- **発音・アクセント**  
  - カタカナの発音が一貫していない（例：「ポスター」→ 「ポスタ」）。  
  - ピッチアクセント（音高の変化）が外国人にとって難しいです

## langchainを使用してみる

In [8]:
messages = [
    HumanMessage(
        content='英語と日本語どちらが難しい？'
    )
]
llm = ChatOpenAI(
    openai_api_key='EMPTY',
    openai_api_base='http://vllm:8000/v1',
    model_name='/models/Qwen3-8B-FP8',
)
chain = (
    llm
    | StrOutputParser()
)
original_result = chain.invoke(
    messages,
    max_tokens=8192,
    top_p=0.8,
    temperature=0.7,
    presence_penalty=1.5,
    extra_body={
        'top_k': 20,
        'chat_template_kwargs': {'enable_thinking': False},
    }
)
result = extract_think_and_others(original_result)
for res in result:
    if res['type'] == 'think':
        continue
    else:
        print(res['content'])

英語と日本語のどちらが難しいかは、**個人の母語や学習経験、目的、そして習得している言語の特徴**によって大きく異なります。以下にそれぞれの難しさを比較する視点を挙げて説明します。

---

## 🌍 **1. 母語との関係**
- **英語を母語とする人（英語話者）にとって：**
  - 日本語は**文法構造や語順が異なる**ため、難しい。
  - 音声（発音）も英語とは異なる（例：「は」や「へ」など）。
  - 語彙も文化的なニュアンスが違う。

- **日本語を母語とする人（日本人）にとって：**
  - 英語は**文法が単純で直訳的**だが、音声やアクセントが複雑。
  - 動詞の活用や時制が英語より複雑。
  - ライティングやスピーキングでは**文化・表現の違い**が障壁になることがある。

---

## 🧠 **2. 文法の難易度**
- **英語の文法：**
  - 動詞の活用（to be, to have, etc.）や時制（過去形、現在進行形など）が複雑。
  - 名詞の可算/不可算、冠詞（a/an/the）の使用が重要。
  - 形容詞や副詞の位置が文の構造に影響。

- **日本語の文法：**
  - 動詞の活用（五段活用、一段活用など）が複雑。
  - 副詞や助詞の使い方が重要。
  - 結論的に「文法が複雑」と感じる人が多い。

---

## 🎵 **3. 音声・発音**
- **英語：**
  - アクセントやイントネーションが重要。
  - カタカナ表記で表される音（例：'th', 'r'）が日本人には難しい。
  - 言語としての音声体系が異なる。

- **日本語：**
  - 母語者が英語を話す際、発音が「外国語っぽい」と感じられやすい。
  - ライティングでは漢字や仮名の読み間違いが起こりやすい。

---

## 📚 **4. 語彙と文化**
- **英語：**
  - 新しい単語や専門用語が多い。
  - 文化的背景が深く、ニュアンスが読解に影響。

- **日本語：**
  - 漢字の読み書きが難しい。
  - 礼儀作法や暗黙の了解がコミュニケーションに影響。

---

## 🧩 **5. 学習目的**
- **英語を学ぶ目的：**
  - 国際交流、ビジネス、留学など。
  - 多くの国で公用語なので、実用性